WITHOUT GUI 

In [1]:
!pip install opencv-python numpy pillow


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   --- ------------------------------------ 3.1/39.5 MB 14.5 MB/s eta 0:00:03
   ----- ---------------------------------- 5.2/39.5 MB 12.7 MB/s eta 0:00:03
   ------- -------------------------------- 7.3/39.5 MB 11.2 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/39.5 MB 11.4 MB/s eta 0:00:03
   ----------- ---------------------------- 11.8/39.5 MB 11.0 MB/s eta 0:00:03
   ------------- -------------------------- 13.6/39.5 MB 10.7 MB/s eta 0:00:03
   --------------- ------------------------ 15.7/39.5 MB 10.6 MB/s eta 0:00:03
   ------------------ --------------------- 17.8/39.5 MB 10.5 MB/s eta 0:00:03
   -------------------- ------------------- 19.9/39.5 MB 10.3 MB/s eta 0:00:02
   --------------------- ------------------ 21.5/39.5 MB 10.2 MB/s eta 0:00:02
   ----------------------- ---------------- 23.6/39.5 MB 10.1 MB/s eta 0:00:02
   -------------------------- ------------- 25.7/39.5 MB 10.1 MB/


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import hashlib
from PIL import Image

def encrypt_image():
    # Ask user for the cover image file
    image_path = input("Enter the path of the cover image (e.g., mypic.jpg): ").strip()
    
    # Ask user for the text file containing the secret message
    text_file = input("Enter the path of the text file containing the secret message: ").strip()
    
    # Ask for password
    password = input("Enter a password for encryption: ").strip()

    # Read the secret message from the text file
    try:
        with open(text_file, "r", encoding="utf-8") as file:
            secret_msg = file.read().strip()
    except FileNotFoundError:
        print("Error: Text file not found!")
        return
    except Exception as e:
        print(f"Error reading the file: {e}")
        return

    # Open image using PIL to preserve metadata
    try:
        img = Image.open(image_path).convert("RGB")  # Ensure RGB mode
        pixels = np.array(img, dtype=np.uint8)  # Ensure proper data type
    except FileNotFoundError:
        print("Error: Image file not found!")
        return
    except Exception as e:
        print(f"Error loading the image: {e}")
        return

    # Check if message can fit in the image
    max_bytes = pixels.shape[0] * pixels.shape[1] * 3 // 8
    secret_msg += "###END###"  # Add a delimiter
    binary_msg = ''.join(format(ord(c), '08b') for c in secret_msg)

    if len(binary_msg) > max_bytes * 8:
        print("Error: Message is too large to hide in this image.")
        return

    # Hash the password (store hashed password for security)
    password_hash = hashlib.sha256(password.encode()).hexdigest()

    # Create a log file to store pixel positions
    log_file = open("pixel_log_without_gui&stealthMode.txt", "w")
    log_file.write("Bit storage locations (row, col, channel):\n")

    # Encode message in Least Significant Bits (LSB)
    idx = 0
    for row in range(pixels.shape[0]):
        for col in range(pixels.shape[1]):
            for channel in range(3):  # Modify R, G, B channels
                if idx < len(binary_msg):
                    pixels[row, col, channel] = (int(pixels[row, col, channel]) & ~1) | int(binary_msg[idx])
                    # Log the position of the bit storage
                    log_file.write(f"Bit {binary_msg[idx]} -> Pixel ({row}, {col}, {channel})\n")
                    idx += 1

    log_file.close()  # Close the log file

    # Save the encrypted image
    encrypted_img = Image.fromarray(pixels)
    encrypted_img.save("encryptedImage__without_gui&stealthMode.png", format="PNG")  # Preserve metadata

    # Save the hashed password for authentication
    with open("password_hash.txt", "w") as f:
        f.write(password_hash)

    print("Message encrypted successfully! Saved as 'encryptedImage_without_gui&stealthMode.png'.")
    print("Pixel positions are saved in 'pixel_log_without_gui&stealthMode.txt'.")

# Run the encryption function
encrypt_image()


Enter the path of the cover image (e.g., mypic.jpg):  Sunflower.webp
Enter the path of the text file containing the secret message:  message.txt
Enter a password for encryption:  1234


Message encrypted successfully! Saved as 'encryptedImage_without_gui&stealthMode.png'.
Pixel positions are saved in 'pixel_log_without_gui&stealthMode.txt'.
